In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy.sparse as sp
from stellargraph import StellarDiGraph

In [15]:
data = pd.read_csv('./England_2009_2010.csv', header=None)
folder_name = './'

In [16]:
data

,0,1,2,3
0,Aston Villa,Wigan,0,2
1,Blackburn,Man City,0,2
2,Bolton,Sunderland,0,1
3,Chelsea,Hull,2,1
4,Everton,Arsenal,1,6
...,...,...,...,...
375,Everton,Portsmouth,1,0
376,Hull,Liverpool,0,0
377,Man United,Stoke,4,0
378,West Ham,Man City,1,1


In [17]:
data2 = data.copy()

In [18]:
data2.columns = [1,0,3,2]

In [19]:
data = pd.concat([data,data2], ignore_index=True)

In [20]:
data

,0,1,2,3
0,Aston Villa,Wigan,0,2
1,Blackburn,Man City,0,2
2,Bolton,Sunderland,0,1
3,Chelsea,Hull,2,1
4,Everton,Arsenal,1,6
...,...,...,...,...
755,Portsmouth,Everton,0,1
756,Liverpool,Hull,0,0
757,Stoke,Man United,0,4
758,Man City,West Ham,1,1


In [21]:
data.columns = ['source', 'target', 'weight', 'lose']
data = data[['source', 'target', 'weight']]

In [22]:
data['weight'] += 0.1
data

/data/localhost/yhe/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,source,target,weight
0,Aston Villa,Wigan,0.1
1,Blackburn,Man City,0.1
2,Bolton,Sunderland,0.1
3,Chelsea,Hull,2.1
4,Everton,Arsenal,1.1
...,...,...,...
755,Portsmouth,Everton,0.1
756,Liverpool,Hull,0.1
757,Stoke,Man United,0.1
758,Man City,West Ham,1.1


In [24]:
graph = StellarDiGraph(edges=data)

In [25]:
print(graph.info())

StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 760

 Node types:
  default: [20]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [760]
        Weights: range=[0.1, 9.1], mean=1.48553, std=1.34254
        Features: none


In [26]:
adj = graph.to_adjacency_matrix(weighted=True)

In [27]:
adj

<20x20 sparse matrix of type '<class 'numpy.float64'>'
	with 380 stored elements in Compressed Sparse Row format>

In [28]:
adj-adj.transpose()

<20x20 sparse matrix of type '<class 'numpy.float64'>'
	with 328 stored elements in Compressed Sparse Row format>

In [29]:
adj.max()

12.2

In [30]:
adj.min()

0.0

In [31]:
node_list = list(graph.nodes())

In [36]:
def football_prepare(dataset):
    data = pd.read_csv('./'+dataset+'.csv', header=None)
    data2 = data.copy()
    data2.columns = [1,0,3,2]
    data = pd.concat([data,data2], ignore_index=True)
    data.columns = ['source', 'target', 'weight', 'lose']
    data = data[['source', 'target', 'weight']]
    data['weight'] += 0.1
    graph = StellarDiGraph(edges=data)
    print(graph.info())
    adj = graph.to_adjacency_matrix(weighted=True)
    assert np.abs(adj-adj.transpose()).sum() != 0
    node_list = list(graph.nodes())
    sp.save_npz('finer' + dataset+'adj', adj)
    np.save('finer' + dataset+'node_list', node_list)
    print(dataset+' done!')

In [37]:
for year in range(2009, 2014):
    dataset = 'England_' + str(year) + '_' + str(year+1)
    football_prepare(dataset)

StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 760

 Node types:
  default: [20]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [760]
        Weights: range=[0.1, 9.1], mean=1.48553, std=1.34254
        Features: none
England_2009_2010 done!
StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 760

 Node types:
  default: [20]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [760]
        Weights: range=[0.1, 7.1], mean=1.49868, std=1.19958
        Features: none
England_2010_2011 done!
StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 760

 Node types:
  default: [20]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [760]
        Weights: range=[0.1, 8.1], mean=1.50263, std=1.2827
        Features: none
England_2011_2012 done!
StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 760

 Node types:
  defau

In [38]:
dataset = 'England_2014_2015'
data = pd.read_csv('./'+dataset+'.csv')
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,16/08/14,Arsenal,Crystal Palace,2,1,H,1,1,D,...,1.77,1.72,2.26,2.10,24,-1.50,1.81,1.78,2.20,2.10
1,E0,16/08/14,Leicester,Everton,2,2,D,1,2,A,...,2.10,2.00,1.90,1.80,22,0.25,1.88,1.85,2.10,2.02
2,E0,16/08/14,Man United,Swansea,1,2,A,0,1,A,...,1.77,1.71,2.30,2.13,25,-1.50,2.18,2.08,1.87,1.79
3,E0,16/08/14,QPR,Hull,0,1,A,0,0,D,...,2.52,2.36,1.65,1.58,24,0.00,1.80,1.73,2.25,2.14
4,E0,16/08/14,Stoke,Aston Villa,0,1,A,0,0,D,...,2.42,2.31,1.67,1.60,23,-0.50,1.95,1.91,2.02,1.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,E0,06/12/14,Stoke,Arsenal,3,2,H,3,0,H,...,2.10,2.00,1.87,1.80,27,0.50,1.93,1.90,2.02,1.97
146,E0,06/12/14,Tottenham,Crystal Palace,0,0,D,0,0,D,...,1.89,1.77,2.12,2.04,31,-1.00,2.12,2.05,1.86,1.82
147,E0,07/12/14,Aston Villa,Leicester,2,1,H,1,1,D,...,2.15,2.06,1.83,1.75,27,-0.25,2.08,2.02,1.90,1.85
148,E0,07/12/14,West Ham,Swansea,3,1,H,1,1,D,...,2.13,2.02,1.85,1.78,27,-0.25,2.08,2.01,1.90,1.86


In [39]:
data = data[['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']]

In [40]:
data

,HomeTeam,AwayTeam,FTHG,FTAG
0,Arsenal,Crystal Palace,2,1
1,Leicester,Everton,2,2
2,Man United,Swansea,1,2
3,QPR,Hull,0,1
4,Stoke,Aston Villa,0,1
...,...,...,...,...
145,Stoke,Arsenal,3,2
146,Tottenham,Crystal Palace,0,0
147,Aston Villa,Leicester,2,1
148,West Ham,Swansea,3,1


In [41]:
data.columns = [0,1,2,3]

In [42]:
data

,0,1,2,3
0,Arsenal,Crystal Palace,2,1
1,Leicester,Everton,2,2
2,Man United,Swansea,1,2
3,QPR,Hull,0,1
4,Stoke,Aston Villa,0,1
...,...,...,...,...
145,Stoke,Arsenal,3,2
146,Tottenham,Crystal Palace,0,0
147,Aston Villa,Leicester,2,1
148,West Ham,Swansea,3,1


In [43]:
data2 = data.copy()
data2.columns = [1,0,3,2]
data = pd.concat([data,data2], ignore_index=True)
data.columns = ['source', 'target', 'weight', 'lose']
data = data[['source', 'target', 'weight']]
data['weight'] += 0.1
graph = StellarDiGraph(edges=data)
print(graph.info())
adj = graph.to_adjacency_matrix(weighted=True)
assert np.abs(adj-adj.transpose()).sum() != 0
node_list = list(graph.nodes())
sp.save_npz('finer' + dataset+'adj', adj)
np.save('finer' + dataset+'node_list', node_list)
print(dataset+' done!')

StellarDiGraph: Directed multigraph
 Nodes: 20, Edges: 300

 Node types:
  default: [20]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [300]
        Weights: range=[0.1, 8.1], mean=1.41, std=1.17694
        Features: none
England_2014_2015 done!
